In [1]:
%cd "path"
from collections import Counter
import re

#topic model packages
import numpy as np
import json
import glob

#Gesnim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy 
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.gensim_models

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\ba3uj5\Dropbox\pol_influencer\Wahlprogramme


C:\Users\ba3uj5\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
C:\Users\ba3uj5\Anaconda3\lib\site-packages\sklearn\decomposition\_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [7]:
from collections import Counter
import re
import nltk
from nltk.collocations import *

In [2]:
#stopwords
!pip install stop-words
from stop_words import get_stop_words

stopwords =get_stop_words("german")
#stopwords = open('stop_words_german.txt', encoding =  'utf-8').readlines()

newset = set()
for s in stopwords:
    newstop = s.replace("\n", "")
    newset.add(newstop)


#define function
def wordfind(input_txt): 
    WP_txt = " ".join(input_txt) 
    WP_onlyword = re.sub(r"[?|$|.|!|,|–]","",WP_txt) 
    split_it = WP_onlyword.split()
    resultwords = [word for word in split_it if word.lower() not in newset]
    Counters_found = Counter(resultwords) 
    most_occur = Counters_found.most_common(100) 
    return(most_occur)#tada!

In [3]:
WP_GR = open('2021_Wahlprogrammentwurf_Gruene.txt', encoding ="utf-8").readlines()
WP_SPD = open('SPD-Zukunftsprogramm.txt', encoding ="utf-8").readlines()
WP_CDU = open('2017_WahlprogrannCDU.txt', encoding ="utf-8").readlines()
WP_FDP = open('2021_Wahlprogramm_FDP.txt', encoding ="utf-8").readlines()
WP_AFD = open('2017-Wahlprogramm_AfD.txt', encoding ="utf-8").readlines()

In [4]:
Topwords_SPD =wordfind(WP_SPD)
Topwords_CDU =wordfind(WP_CDU)
Topwords_AFD =wordfind(WP_AFD)
Topwords_FDP =wordfind(WP_FDP)
Topwords_GR = wordfind(WP_GR)

In [5]:
print(Topwords_FDP)
with open('TopwordsFDP.json', 'w', encoding = "utf-8") as fp:
    json.dump(Topwords_FDP, fp)

[('Freie', 347), ('Demokraten', 345), ('fordern', 99), ('müssen', 98), ('Deutschland', 96), ('mehr', 91), ('Menschen', 61), ('sollen', 61), ('setzen', 60), ('stärken', 58), ('sowie', 56), ('schaffen', 52), ('EU', 51), ('Unternehmen', 50), ('ermöglichen', 42), ('Bildung', 39), ('Chancen', 39), ('fördern', 34), ('beim', 33), ('ab', 33), ('Freiheit', 32), ('europäischen', 32), ('Zukunft', 31), ('Schutz', 31), ('neue', 30), ('Wettbewerb', 29), ('Ziel', 29), ('Dabei', 27), ('dafür', 26), ('Bürgerinnen', 26), ('brauchen', 26), ('unterstützen', 26), ('Staat', 25), ('Europa', 25), ('verhindern', 25), ('insbesondere', 25), ('daher', 25), ('Digitalisierung', 24), ('besser', 24), ('einführen', 24), ('digitalen', 24), ('Zudem', 24), ('Bürger', 23), ('digitale', 23), ('lehnen', 23), ('erhalten', 23), ('darf', 22), ('immer', 22), ('stellen', 22), ('Politik', 21), ('stärker', 21), ('abschaffen', 21), ('Altersvorsorge', 21), ('bleiben', 20), ('Land', 20), ('Selbstbestimmung', 20), ('Gesellschaft', 20)

In [6]:
#zur Kontrolle wieder öffnen
with open ("TopwordsFDP.json", "r", encoding = "utf-8") as f:
    FDP = json.load(f)


In [8]:
#define function for collocations
def collocation_opt(input_txt): 
    WP_txt = " ".join(input_txt) 
    WP_onlyword = re.sub(r"[?|$|.|!|,|-|/|-|:]","",WP_txt)
    WP_onlyword = WP_onlyword.replace("*","")
    WP_onlyword = WP_onlyword.replace("-","")
    split_it = WP_onlyword.split()
    resultwords = [word for word in split_it if word.lower() not in newset]
    Resultwords = [x.lower() for x in resultwords]
    resultstring = " ".join(Resultwords)
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    trigram_measures = nltk.collocations.TrigramAssocMeasures()
    tokens = nltk.wordpunct_tokenize(resultstring)
    finder = BigramCollocationFinder.from_words(tokens)
    scored = finder.score_ngrams(bigram_measures.raw_freq)
    sorted(bigram for bigram, score in scored)  # doctest:
    return scored


In [9]:
#convert txt to usefull string
def prettytext(input_txt):
    WP_txt = " ".join(input_txt) 
    WP_onlyword = re.sub(r"[?|$|.|!|,|-|/|–|:]","",WP_txt)
    WP_onlyword = WP_onlyword.replace("*","") #Gendersternchen
    split_it = WP_onlyword.split()
    resultwords = [word for word in split_it if word.lower() not in newset]
    Resultwords = [x.lower() for x in resultwords]
    return Resultwords

GR_pretty = prettytext(WP_GR)
#GR_pretty[:100]

In [10]:
#count collcations
def collocation(input_txt): 
    WP_txt = " ".join(input_txt) 
    WP_onlyword = re.sub(r"[?|$|.|!|,|-|/|–|:]","",WP_txt)
    WP_onlyword = WP_onlyword.replace("*","")
    split_it = WP_onlyword.split()
    resultwords = [word for word in split_it if word.lower() not in newset]
    Resultwords = [x.lower() for x in resultwords]
    resultstring = " ".join(Resultwords)
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    trigram_measures = nltk.collocations.TrigramAssocMeasures()
    tokens = nltk.wordpunct_tokenize(resultstring)
    finder = BigramCollocationFinder.from_words(tokens)
    scored = finder.score_ngrams(bigram_measures.raw_freq)
    sorted(bigram for bigram, score in scored) 
    

In [11]:
#lemmatize

def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


In [12]:
lemmatized_texts = lemmatization(GR_pretty)
print (lemmatized_texts[0:90])


['deutschland', 'drin', 'programmentwurf', 'bundestagswahl', '', 'inhalt', '', 'einladung', '', 'lebensgrundlagen', 'schützen', '', 'schaffen', 'klimagerechten', 'wohlstand', '', 'schaffen', 'versorgungssicherheit', 'erneuerbaren', '', 'sorgen', 'nachhaltige', 'mobilität', '', 'schützen', 'natur', 'umwelt', 'gute', 'leben', '', 'stärken', 'bäuerinnen', 'tiere', 'natur', '', 'ermöglichen', 'tieren', 'bessere', 'leben', '', 'zukunft', 'wirtschaften', '', 'fördern', 'unternehmergeist', 'wettbewerb', 'ideen', '', 'geben', 'markt', 'sozial ökologischen', 'rahman', '', 'bringen', 'digitalisierung', 'voran', '', 'kämpfen', 'fairen', 'nachhaltigen', 'handel', '', 'finanzmärkte', 'stabiler', 'nachhaltiger', '', 'vollenden', 'europäische', 'wirtschafts-', 'währungsunion', '', 'haushalten', 'solide', 'weitsichtig', 'gerecht', '', 'solidarität', 'sichern', '', 'fördern', 'kind', 'jugendliche', 'familien', '', 'sorgen', 'gute', 'arbeit', '', 'löhne', '']


In [13]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0:20])

[['deutschland'], ['drin'], ['programmentwurf'], ['bundestagswahl'], [], ['inhalt'], [], ['einladung'], [], [], ['schutzen'], [], ['schaffen'], ['klimagerechten'], ['wohlstand'], [], ['schaffen'], [], ['erneuerbaren'], []]


In [14]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:5][0]]
print (word)

[(0, 1)]
deutschland


In [16]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

C:\Users\ba3uj5\Anaconda3\lib\site-packages\sklearn\manifold\_mds.py:128: RuntimeWarning: divide by zero encountered in double_scalars
  old_stress = stress / dis
C:\Users\ba3uj5\Anaconda3\lib\site-packages\sklearn\manifold\_mds.py:123: RuntimeWarning: invalid value encountered in double_scalars
  if(old_stress - stress / dis) < eps:
C:\Users\ba3uj5\Anaconda3\lib\site-packages\sklearn\manifold\_mds.py:128: RuntimeWarning: invalid value encountered in double_scalars
  old_stress = stress / dis
C:\Users\ba3uj5\Anaconda3\lib\site-packages\sklearn\manifold\_mds.py:123: RuntimeWarning: invalid value encountered in double_scalars
  if(old_stress - stress / dis) < eps:
C:\Users\ba3uj5\Anaconda3\lib\site-packages\sklearn\manifold\_mds.py:128: RuntimeWarning: invalid value encountered in double_scalars
  old_stress = stress / dis
C:\Users\ba3uj5\Anaconda3\lib\site-packages\sklearn\manifold\_mds.py:123: RuntimeWarning: invalid value encountered in double_scalars
  if(old_stress - stress / dis) <

PreparedData(topic_coordinates=         x    y  topics  cluster      Freq
topic                                     
25     0.0  0.0       1        1  3.576523
21     0.0  0.0       2        1  3.324972
0      0.0  0.0       3        1  3.324947
1      0.0  0.0       4        1  3.324947
28     0.0  0.0       5        1  3.324947
27     0.0  0.0       6        1  3.324947
26     0.0  0.0       7        1  3.324947
24     0.0  0.0       8        1  3.324947
23     0.0  0.0       9        1  3.324947
22     0.0  0.0      10        1  3.324947
20     0.0  0.0      11        1  3.324947
19     0.0  0.0      12        1  3.324947
18     0.0  0.0      13        1  3.324947
17     0.0  0.0      14        1  3.324947
16     0.0  0.0      15        1  3.324947
15     0.0  0.0      16        1  3.324947
14     0.0  0.0      17        1  3.324947
13     0.0  0.0      18        1  3.324947
12     0.0  0.0      19        1  3.324947
11     0.0  0.0      20        1  3.324947
10     0.0  0.0      21        1  3.324947
9      0.0  0.0      22        1  3.324947
8      0.0  0.0      23        1  3.324947
7      0.0  0.0      24        1  3.324947
6      0.0  0.0      25        1  3.324947
5      0.0  0.0      26        1  3.324947
4      0.0  0.0      27        1  3.324947
3      0.0  0.0      28        1  3.324947
2      0.0  0.0      29        1  3.324947
29     0.0  0.0      30        1  3.324947, topic_info=                 Term      Freq     Total Category  logprob  loglift
0         deutschland  3.000000  3.000000  Default  30.0000     30.0
4623  forschungsgeist  3.000000  3.000000  Default  29.0000     29.0
4634         unnotige  3.000000  3.000000  Default  28.0000     28.0
4633         optimale  3.000000  3.000000  Default  27.0000     27.0
4632     ausprobieren  3.000000  3.000000  Default  26.0000     26.0
...               ...       ...       ...      ...      ...      ...
25       wirtschaften  0.111946  3.366859  Topic30  -8.8451      0.0
26            fordern  0.111946  3.366859  Topic30  -8.8451      0.0
27         wettbewerb  0.111946  3.366859  Topic30  -8.8451      0.0
28              ideen  0.111946  3.366859  Topic30  -8.8451      0.0
29              geben  0.111946  3.366859  Topic30  -8.8451      0.0

[930 rows x 6 columns], token_table=Empty DataFrame
Columns: [Topic, Freq, Term]
Index: [], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[26, 22, 1, 2, 29, 28, 27, 25, 24, 23, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 30])